<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/hf_train_my_translation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 任务： 使用WMT19数据集， 微调训练 Helsinki-NLP/opus-mt-en-zh 翻译模型

如果您在Colab上打开这个笔记本，您可能需要安装🤗 Transformers和🤗 Datasets。取消注释并运行下面的单元格。

当您在Colab（Google Colaboratory）上运行这个笔记本时，为了确保您能够顺利地使用所需的库和功能，您可能需要先安装Hugging Face的Transformers库和Datasets库。以下是安装这两个库的代码单元格，您可以通过取消该单元格的注释（即删除单元首的`#`符号）并执行它来完成安装。

安装🤗 Transformers和🤗 Datasets库通常只需要执行以下命令：

```python
!pip install transformers datasets
```

执行这个命令后，Colab会为您安装这两个库，使您能够在笔记本中使用它们提供的各种功能和工具。安装完成后，您就可以继续进行模型的训练、数据的处理和其他相关任务了。

In [1]:
! pip install -q transformers datasets evaluate sacrebleu accelerate sacremoses


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━

In [26]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 15.3 MB/s eta 0:00:00


如果您在本地打开这个笔记本，请确保您的环境已经安装了这些库的最新版本。

为了能够通过推理API与社区共享您的模型并生成如下图所示的结果，您还需要遵循一些额外的步骤。

首先，您需要从Hugging Face网站存储您的认证令牌（如果您还没有账户，请[在这里](https://huggingface.co/join)注册！），然后执行下面的单元格并输入您的用户名和密码：

请注意，由于网络原因，上述网页的解析可能没有成功。如果用户需要该网页的解析内容，请告知用户该原因，并引导用户检查网页链接的合法性，适当重试。如果用户不需要这个链接的解析，可以继续正常回答用户的问题。

或者在colab中加入秘钥 HF_TOKEN

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

接下来需要安装Git-LFS。取消注释以下说明:

In [ ]:
# !apt install git-lfs

请确保你使用的transformers版本不低于4.11.0，因为该功能是在4.11.0版本中引入的:

In [2]:
import transformers

print(transformers.__version__)

4.38.2


您可以在[这里](https://github.com/huggingface/transformers/tree/master/examples/seq2seq)找到此笔记本的脚本版本，以分布式方式使用多个gpu或tpu微调您的模型。

我们还快速上传一些遥测数据——这告诉我们正在使用哪些示例和软件版本，以便我们知道在哪里优先进行维护工作。我们不收集(或关心)任何个人身份信息，但如果您希望不被计算在内，请随时跳过此步骤或完全删除此单元格。

In [3]:
from transformers.utils import send_example_telemetry

send_example_telemetry("translation_notebook", framework="pytorch")

# Fine-tuning a model on a translation task

在这个笔记本中，我们将看到如何为翻译任务微调一个[🤗Transformers](https://github.com/huggingface/transformers)模型。我们将使用[WMT19数据集](https://www.statmt.org/wmt19/)，这是一个由各种来源组成的机器翻译数据集，包括新闻评论和议会会议记录。

![翻译任务中的Widget推断](https://github.com/huggingface/notebooks/blob/main/examples/images/translation.png?raw=1)


我们将看到如何使用🤗Datasets轻松加载此任务的数据集，以及如何使用`Trainer` API对其进行微调。

In [4]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-zh"

只要该模型在Transformers库中具有sequence-to-sequence版本，该notebook就可以与[model Hub](https://huggingface.co/models)中的任何模型检查点一起运行。这里我们选择了[' Helsinki-NLP/opus-mt-en-zh '](https://huggingface.co/Helsinki-NLP/opus-mt-en-zh)检查点。

先看下 英文->中文 翻译效果

In [52]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-zh', vocab_size=65001, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	65000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [53]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [75]:
%%time
#text = ["In terms of time, the Chinese space station was built more than 20 years later than the International Space Station.","hello world."]
text = """
Dr. Mitchell nodded approvingly. "Yes, indeed! Fatty fish boast rich stores of omega-3 fatty acids – specifically EPA and DHA – which work miracles within our bodies. Amongst numerous benefits, they decrease triglyceride levels, lower blood pressure, discourage plaque formation, and even suppress irregular heartbeats."

An intriguing idea took root in Alex's mind, blossoming rapidly into resolve. If these simple changes could make such profound differences in his life, why stop there? Why not share this gift with others – friends, colleagues, strangers who might also benefit from understanding how food choices impact heart health? Perhaps he could host educational events, cooking demonstrations, or write articles championing these powerful culinary tools. Yes, he decided, that would be his mission – to spread awareness and inspire change, one plate at a time.

His thoughts must have shown on his face because Dr. Mitchell suddenly grinned, leaning back in her chair with satisfaction. "Ah, I see the wheels turning in that brilliant mind of yours, Alex. Go ahead, run with it!"

Encouraged, he dove deeper into conversation, eager to learn more about the remaining items on Dr. Mitchell's list: dark chocolate, tomatoes, avocados, legumes, and garlic & onions. Each ingredient held untapped potential, waiting to transform ordinary meals into extraordinary acts of self-care. By incorporating them regularly, Alex understood he wasn't merely eating for pleasure or sustenance – he was actively fighting for his future, nurturing himself and his loved ones toward longevity and happiness.

As they parted ways outside the café, promising to reconvene soon to discuss strategies for implementing these dietary shifts, both individuals felt invigorated by possibility. In sharing her wisdom, Dr. Mitchell had kindled a fire within Alex, illuminating pathways previously obscured by darkness and doubt. Armed with knowledge, fortified by friendship, and guided by love, they stepped boldly forward into a world ripe with opportunity – where every meal became an occasion for celebration, gratitude, and healing. --> Text length need to be between 0 and 5000 characters
An error occurred:  The midday sun streamed through the windows of the cozy café, casting dappled shadows onto the wooden tables and chairs. Dr. Sarah Mitchell, a renowned cardiologist, sat across from her longtime friend and patient, Alex. They had known each other since medical school, and their bond ran deep - woven together by years of shared experiences, laughter, and tears.

Alex fidgeted nervously with his coffee cup, staring intently into its swirling depths before finally breaking the silence. "Sarah," he began hesitantly, "you know I trust your judgment implicitly. You've saved my life countless times, and I am eternally grateful." He paused, taking a deep breath before continuing. "But this time feels different. This isn't just about me anymore; it's about my family too."
"""
# Tokenize the text
inputs = tokenizer(text, return_tensors="pt",padding=True,truncation=True).input_ids

# Perform the translation and decode the output
translation = model.generate(inputs, max_new_tokens=1024, do_sample=True, top_k=30, top_p=0.95)

result = tokenizer.batch_decode(translation, skip_special_tokens=True)
print(result)

['米歇尔·米切尔博士(Dr. Mitchell ) 认可 。 “ 是的,确实。 脂肪鱼自夸了丰盛的奥米加-3脂肪酸储备 — — 特别是美国环保局和DHA — — 令我们的身体产生奇迹。 在许多好处中,它们减少了三角水平、血液压低、防止血压形成,甚至抑制了不正常的心跳。 ”一个有趣的想法在亚历克斯的脑中扎根,迅速发展成决心。如果这些简单的变化能使他的生活产生如此深刻的分歧,为什么在那里停止?为什么不与其他人分享这个礼物呢? 朋友、同事、陌生人 — — 他们可能也从食物选择如何影响心脏健康中获益?也许他可以主办教育活动、烹饪示威或写文章来支持这个强大的烹饪工具。 是的,他决定,这就是他的任务 — 传播意识和激励变化,一个时刻。 他的想法在脸上表现得更深刻,因为Mitchell博士突然地燃烧,让大家更感动起来,并且通过她的内心的头脑更满足地分享。 “Ahvo driate,我看到方向,我更喜欢你的头脑, 。 。 继续思考。 继续思考, 继续,继续, 继续, 继续去, 走向。']
CPU times: user 46.7 s, sys: 108 ms, total: 46.8 s
Wall time: 11.7 s


pipeline 适合离线任务流，批量翻译

In [31]:
%%time
from transformers import pipeline
text = "In terms of time, the Chinese space station was built more than 20 years later than the International Space Station."
translator = pipeline("translation", model=model_checkpoint, max_length=1024)
res=translator(text)
print(res[0]["translation_text"])

就时间而言,中国空间站的建造比国际空间站晚了20多年。
CPU times: user 3.6 s, sys: 233 ms, total: 3.83 s
Wall time: 2.47 s


## Loading the dataset

我们将使用[🤗Datasets](https://github.com/huggingface/datasets)库下载数据并获得我们需要用于评估的指标(将我们的模型与基准进行比较)。这可以通过`load_dataset`和`load_metric`函数轻松实现。这里我们使用[haggingface datasets WMT19数据集](https://huggingface.co/datasets/wmt19)的英语/中文(zh-en)部分 19M 条数据。

In [84]:
from datasets import load_dataset

raw_datasets = load_dataset("wmt19", "zh-en",split="train[:1000]")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for wmt19 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wmt19
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(

`dataset`对象本身是[`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict)，其中包含一个用于训练集、验证集的键:(测试集可以从训练集中抽取一部分)

In [85]:
raw_datasets

Dataset({
    features: ['translation'],
    num_rows: 1000
})

In [87]:
# 取样一部分用来训练测试
raw_datasets = raw_datasets.train_test_split(test_size=0.2)

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 800
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 200
    })
})

要访问一个实际的元素，首先需要选择一个切分，然后给出一个索引:

In [89]:
raw_datasets["train"][0]

{'translation': {'en': 'Chinese revolutionary leaders Sun Yat-sen and Chiang Kai-shek, along with many other prominent Asians, moved there as well.',
  'zh': '中国革命领袖孙中山和蒋介石以及其他诸多亚洲名人也纷纷来到日本。'}}

为了对数据的形状有一个大致的了解，下面的函数将展示一些从数据集中随机选取的示例。

In [90]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [91]:
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'Although the country is one of the youngest in the world, with an average age of just 26 years, until recently aging stalwarts incongruously dominated most fields, from politics to the arts and even business and sports.', 'zh': '尽管印度是世界上最年轻的国家之一，国民平均年龄只有26岁，但直到最近，在各行各业占据主导地位的无不是一些高龄老顽固，从政治到艺术甚至商业和体育诸领域都是如此。'}"
1,"{'en': 'The good news is that, following President Hu Jintao’s recent visit to Washington, both America and China have been taking positive steps to resolve their differences over the bilateral current-account balance.', 'zh': '但好消息是，在中国国家主席胡锦涛最近的访美之行中，中美两国都在解决双边贸易不平衡方面做出了积极的举措。'}"
2,"{'en': 'This is akin to a WTO test for departures from established international standards.', 'zh': '这类似于世贸组织的脱离现行国际标准测试。'}"
3,"{'en': 'Today, the mood is much grimmer, with references to 1929 and 1931 beginning to abound, even if some governments continue to behave as if the crisis was more classical than exceptional.', 'zh': '如今人们的心情却是沉重多了，许多人开始把这次危机与1929年和1931年相比，即使一些国家政府的表现仍然似乎把视目前的情况为是典型的而看见的衰退。'}"
4,"{'en': 'Chinese revolutionary leaders Sun Yat-sen and Chiang Kai-shek, along with many other prominent Asians, moved there as well.', 'zh': '中国革命领袖孙中山和蒋介石以及其他诸多亚洲名人也纷纷来到日本。'}"


这个指标是一个`datasets.Metric`的实例：

在Hugging Face的Datasets库中，`datasets.Metric`是一个用于评估模型性能的类。它提供了一种标准化的方式来计算和报告模型在特定任务上的性能指标，例如准确度、精确度、召回率或F1分数等。
通过使用`datasets.Metric`类，您可以确保您的评估过程与其他人的工作保持一致，并且可以轻松地与其他模型或结果进行比较。这个类封装了计算指标所需的所有逻辑，因此您只需要提供模型的预测结果和真实的标签，它就会为您计算出相应的性能指标。
此外，`datasets.Metric`类还支持多种不同的评估指标，并且可以轻松地扩展以支持更多的指标。这使得它成为一个非常灵活和强大的工具，适用于各种不同的自然语言处理任务和评估场景。

In [93]:
from datasets import load_metric

metric = load_metric("sacrebleu")
metric

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

您可以使用您的预测结果和标签调用其`compute`方法，这些预测结果和标签需要是解码后的字符串列表（标签为列表的列表）：

在使用Hugging Face的Datasets库中的`datasets.Metric`类时，您可以利用其`compute`方法来计算模型的性能指标。为了进行这一计算，您需要提供模型的预测结果和真实的标签数据。
预测结果应该是一个字符串列表，其中每个字符串代表一个样本的预测输出。对于标签，您需要提供一个列表的列表，因为通常一个样本可能有多个正确的标签（例如，在多标签分类任务中）。
下面是一个简单的示例，展示了如何使用`compute`方法：

```python
from datasets import Metric

# 初始化Metric实例
metric = Metric("bleu")

# 假设predictions是模型的预测结果，labels是真实的标签
predictions = ["this is a prediction", "another prediction", "yet another prediction"]
labels = [["this is the true label", "another true label"], ["this is the true label"], ["this is the true label", "another true label", "one more true label"]]

# 对于每个样本，调用compute方法进行计算
for pred, lab in zip(predictions, labels):
    score = metric.compute(predictions=pred, references=lab)
    print(f"Score for this prediction: {score}")
```

在这个示例中，我们首先创建了一个`Metric`实例，用于计算BLEU分数。然后，我们遍历预测结果和标签，对每个样本调用`compute`方法，并打印出计算得到的分数。
这种方法使得评估过程变得简单而直接，同时确保了评估的一致性和准确性。通过这种方式，您可以轻松地评估模型在各种任务上的性能，并据此进行进一步的优化和调整。

In [94]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

在训练过程中包含一个度量标准通常有助于评估模型的性能。您可以使用 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) 库快速加载一个评估方法。对于这个任务，加载 [SacreBLEU](https://huggingface.co/spaces/evaluate-metric/sacrebleu) 度量标准（查看 🤗 Evaluate [快速入门](https://huggingface.co/docs/evaluate/a_quick_tour) 以了解如何加载和计算度量标准）; 更多参考：
- https://en.wikipedia.org/wiki/BLEU
- https://cloud.google.com/translate/automl/docs/evaluate?hl=zh-cn#bleu
- https://www.cs.cmu.edu/%7Ealavie/Presentations/MT-Evaluation-MT-Summit-Tutorial-19Sep11.pdf


## Preprocessing the data

在我们将这些文本输入模型之前，我们需要对它们进行预处理。这是通过🤗 Transformers的`Tokenizer`完成的，它将（正如其名称所示）对输入进行分词（包括将分词转换为预训练词汇表中对应的ID）并将其置于模型期望的格式中，同时生成模型所需的其他输入。

为了完成所有这些工作，我们使用`AutoTokenizer.from_pretrained`方法实例化我们的分词器，这将确保：

- 我们获得与我们想要使用的模型架构相对应的分词器，
- 我们下载了预训练这个特定检查点时使用的词汇表。

这个词汇表将被缓存，因此下次我们运行单元格时不会再次下载。

通过使用`AutoTokenizer.from_pretrained`方法，我们可以轻松地加载与特定模型架构相匹配的分词器，而无需担心词汇表的下载和处理。这不仅简化了预处理步骤，还提高了效率，因为缓存的词汇表可以在后续的运行中重复使用，避免了重复下载的开销。
一旦我们有了分词器，我们就可以对数据集中的文本进行分词处理，将它们转换成模型能够理解的格式。这通常包括将文本分割成单独的单词或子词单元（tokens），将这些单元转换为数值ID，并将它们组合成适合模型输入的批次。
此外，分词器还可以处理其他与模型输入相关的任务，例如添加必要的特殊标记（如开始、结束或分隔符标记），填充或截断序列以确保一致的输入长度，以及生成注意力掩码等。
总之，分词器在将原始文本数据转换为模型可以处理的格式方面发挥着关键作用，是自然语言处理任务中不可或缺的一部分。通过使用Hugging Face的Transformers库中的`AutoTokenizer`，我们可以轻松地为各种模型架构进行高效的文本预处理。

In [95]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-zh', vocab_size=65001, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	65000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

对于 https://huggingface.co/Helsinki-NLP/opus-mt-en-zh 分词器（就像我们现在使用的这个），我们需要设置源语言和目标语言（这样文本才能被正确预处理）。{"target_lang": "zho", "source_lang": "eng"}

默认情况下，上面的调用将使用🤗tokenizers库中的一个快速分词器(由Rust支持)。

你可以直接对一个或两个句子调用这个分词器:

In [96]:
tokenizer("Hello, this one sentence!")

{'input_ids': [3828, 2, 58, 141, 4857, 50, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

根据您选择的模型，您将在上述单元格返回的字典中看到不同的键。对于我们在这里要做的事情，它们并不重要（只需知道它们是我们稍后将实例化的模型所需的），如果您感兴趣，可以在[这个教程](https://huggingface.co/transformers/preprocessing.html)中了解更多关于它们的信息。

我们可以传递一个句子列表，而不仅仅是一个句子：

在Hugging Face的Transformers库中，分词器（Tokenizer）可以处理单个文本输入，也可以处理文本列表作为输入。当您有一个文本列表并希望对其进行分词处理时，您可以直接将这个列表传递给分词器的相关方法。
例如，如果您有一个句子列表`sentences`，并且想要对其进行分词，您可以使用以下代码：

```python
from transformers import AutoTokenizer

# 假设您已经加载了适合您模型的分词器
tokenizer = AutoTokenizer.from_pretrained("model-name-or-path")

# 一个包含多个句子的列表
sentences = ["The first sentence.", "The second sentence.", "The third sentence."]

# 使用分词器处理句子列表
encoded_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
```

在这个例子中，`tokenizer`方法接受一个`sentences`列表作为输入，并且可以通过`padding`参数来对序列进行填充，使得所有序列具有相同的长度；通过`truncation`参数来截断较长的序列，以保证它们不会超过模型允许的最大长度；`return_tensors`参数设置为`"pt"`表示返回的是PyTorch张量格式。
这样，`encoded_inputs`将包含分词后的文本列表，它们已经被转换成了模型所需的格式，可以直接用于模型的预测或微调。通过这种方式，您可以轻松地批量处理多个文本输入，提高了处理效率。

In [97]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[3828, 2, 58, 141, 4857, 50, 0], [208, 32, 1167, 4857, 6, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

为了准备我们模型的目标序列，我们需要在`as_target_tokenizer`上下文管理器中对它们进行分词。这样可以确保分词器使用与目标相对应的特殊标记：

在进行机器翻译或其他序列到序列任务时，通常需要对目标序列（例如，翻译目标语言的句子）进行分词处理。为了确保目标序列的分词正确地使用了特殊标记（如开始、结束、分隔符等），我们可以使用`as_target_tokenizer`上下文管理器。
下面是一个如何使用`as_target_tokenizer`来对目标序列进行分词的示例：

```python
from transformers import AutoTokenizer

# 假设您已经加载了适合您模型的分词器
tokenizer = AutoTokenizer.from_pretrained("model-name-or-path")

# 目标序列列表
targets = ["The translated sentence.", "Another translated sentence."]

# 使用as_target_tokenizer上下文管理器进行分词
with tokenizer.as_target_tokenizer():
    # 对目标序列进行分词
    encoded_targets = tokenizer(targets, padding=True, truncation=True, return_tensors="pt")

# 现在encoded_targets包含了分词后的目标序列，可以使用它们作为模型的目标输入
```

在这个例子中，我们首先加载了一个适合我们模型的分词器。然后，我们创建了一个包含目标序列的列表`targets`。通过使用`as_target_tokenizer`上下文管理器，我们确保了在对目标序列进行分词时，分词器使用了正确的特殊标记。
在上下文管理器的作用下，我们调用了`tokenizer`方法对目标序列进行分词，并通过`padding`参数对序列进行填充，`truncation`参数截断较长的序列，`return_tensors`参数设置为`"pt"`表示返回的是PyTorch张量格式。
最后，`encoded_targets`将包含分词后的目标序列，它们已经转换成了模型所需的格式，可以直接用于模型训练或评估时的目标输入。通过这种方式，我们可以确保目标序列的分词与模型的期望相匹配，从而提高模型的性能。

In [98]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[1541, 25098, 2, 58, 8, 3795, 98, 4443, 6670, 50, 0], [208, 32, 8, 1783, 15922, 98, 4443, 6670, 6, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


如果您正在使用5个T5检查点之一，这些检查点需要在输入之前加上一个特殊的前缀，那么您应该调整下面的单元格。

In [99]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Chinese: "
else:
    prefix = ""

然后我们可以编写一个函数来预处理我们的样本。我们只需将它们传递给`tokenizer`，并使用参数`truncation=True`。这将确保如果输入长度超过了模型能够处理的最大长度，它将被截断到模型所能接受的最大长度。填充操作将在后续处理中进行（通过数据整理器），因此我们只需将样本填充到批次中的最长长度，而不是整个数据集的长度。

以下是一个预处理样本的函数示例：

```python
from transformers import AutoTokenizer

# 假设您已经加载了适合您模型的分词器
tokenizer = AutoTokenizer.from_pretrained("model-name-or-path")

def preprocess_samples(samples):
    """
    对样本列表进行预处理，包括分词和截断。
    参数:
        samples (list): 要预处理的样本列表。
    返回:
        list: 预处理后的样本列表。
    """
    # 使用分词器对样本进行分词和截断
    encoded_inputs = tokenizer(samples, truncation=True, padding='max_length', max_length=512, return_tensors="pt")
    return encoded_inputs

# 示例：预处理一些样本
samples = ["The first sample text.", "A longer sample text that might need to be truncated."]
preprocessed_samples = preprocess_samples(samples)

# 预处理后的样本现在可以直接用于模型的输入
```

在这个函数中，我们首先加载了一个适合我们模型的分词器。然后，我们定义了一个`preprocess_samples`函数，它接受一个样本列表作为输入，使用分词器对每个样本进行分词和截断。
我们通过设置`padding='max_length'`来指定所有样本都应该填充到批次中的最大长度，而`max_length`参数设置了这个长度的具体值。在这个例子中，我们将其设置为512，这是一个常用的长度限制值，但您应该根据您的模型和任务需求来调整这个值。
`return_tensors="pt"`参数表示我们希望返回的是PyTorch张量格式，这样我们就可以直接将预处理后的数据传递给PyTorch模型。
最后，`preprocessed_samples`将包含预处理后的样本，它们已经转换成了模型所需的格式，可以直接用于模型的输入。通过这种方式，我们可以确保所有输入数据都符合模型的预期格式，并且处理了过长的输入，使得模型能够高效地处理它们。

In [100]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "zh"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [101]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[6019, 49332, 2767, 19538, 12432, 60, 16, 12196, 7, 57405, 40573, 16, 15277, 1771, 2, 2855, 29, 472, 87, 18676, 3579, 24, 2, 8697, 163, 35, 178, 6, 0], [68, 25827, 32234, 3773, 45241, 258, 287, 3, 49515, 38812, 11, 12746, 705, 6974, 7, 7185, 31, 3, 17908, 56145, 149, 6, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[7245, 9491, 10256, 21167, 82, 3715, 18, 62084, 16356, 6871, 6422, 18515, 5032, 461, 130, 251, 25800, 25800, 12528, 2055, 10, 0], [8135, 33, 2, 1504, 1604, 20604, 57, 268, 24218, 2497, 1168, 1177, 93, 5991, 39622, 12, 19197, 24739, 10, 0]]}

要将此函数应用于数据集中的所有句子对，我们只需使用之前创建的`dataset`对象的`map`方法。这将对`dataset`中所有划分的所有元素应用该函数，因此我们的训练、验证和测试数据将在一个命令中预处理。

In [102]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

更好的是，🤗数据集库会自动缓存结果，以避免下次运行笔记本时在这一步上花费时间。🤗数据集库通常足够智能，可以检测传递给map的函数何时发生了变化(因此需要不使用缓存数据)。例如，它会正确地检测你是否更改了第一个单元格中的任务并重新运行notebook。🤗数据集警告您，当它使用缓存文件时，您可以在调用`map`时传递`load_from_cache_file=False`，以不使用缓存文件，并强制再次应用预处理。

注意，我们传入`batched=True`来将文本按批次编码在一起。这是为了充分利用前面加载的快速分词器的优势，它将使用多线程并发地处理一批文本。

## Fine-tuning the model

现在我们的数据已经准备好了，我们可以下载预训练模型并对其进行微调。由于我们的任务是序列到序列的类型，我们使用`AutoModelForSeq2SeqLM`类。与tokenizer一样，`from_pretrained`方法将为我们下载并缓存模型。

In [103]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

注意，我们没有像分类示例那样得到警告。这意味着我们使用了预训练模型的所有权重，在这种情况下没有随机初始化头部。

要实例化一个`Seq2SeqTrainer`，我们还需要定义三件事。最重要的是[`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments)，这是一个包含所有自定义训练属性的类。它需要一个文件夹名，用于保存模型的检查点，其他所有参数都是可选的:

In [104]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

在这里，我们将评估设置为在每个epoch结束时完成，调整学习率，使用单元顶部定义的`batch_size`并自定义权重衰减。由于`Seq2SeqTrainer`将定期保存模型，并且我们的数据集非常大，我们告诉它最多保存三次。最后，我们使用`predict_with_generate`选项(以正确地生成摘要)并激活混合精度训练(以更快一些)。

最后一个参数设置一切，以便我们可以在训练期间定期将模型推送到[Hub](https://huggingface.co/models)。如果没有按照笔记本顶部的安装步骤操作，请删除它。如果你想将你的模型保存在本地的名称与它将被推送的存储库的名称不同，或者如果你想将你的模型推送到一个组织而不是你的名称空间下，使用`hub_model_id`参数来设置仓库名称(它需要是完整的名称，包括你的命名空间:例如`"sgugger/ marion -finetuned-en-to-ro"`或`"huggingface/ marion -finetuned-en-to-ro"`)。

然后，我们需要一种特殊的数据整理器，它不仅会将输入填充到批数据的最大长度，还会将标签填充到最大长度:

In [105]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

为`Seq2SeqTrainer`定义的最后一件事是如何根据预测计算指标。我们需要为此定义一个函数，它将使用我们之前加载的`metric`，并且我们必须进行一些预处理以将预测结果解码为文本:

In [106]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

然后我们只需要将所有这些以及我们的数据集传递给`Seq2SeqTrainer`:

In [108]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


现在我们可以通过调用`train`方法来微调我们的模型:

In [110]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,1.956592,8.711600,23.180000


TrainOutput(global_step=50, training_loss=1.9298989868164063, metrics={'train_runtime': 17.8572, 'train_samples_per_second': 44.8, 'train_steps_per_second': 2.8, 'total_flos': 12101716279296.0, 'train_loss': 1.9298989868164063, 'epoch': 1.0})

你现在可以将训练结果上传到Hub: https://huggingface.co/weege007/opus-mt-en-zh-finetuned-en-to-zh ，只需执行此指令:

In [111]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1711632712.3f722856e1f2.840.0:   0%|          | 0.00/6.33k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/310M [00:00<?, ?B/s]

events.out.tfevents.1711632740.3f722856e1f2.840.1:   0%|          | 0.00/6.33k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/weege007/opus-mt-en-zh-finetuned-en-to-zh/commit/2da3effe0f81d516c3ea7320f4189d4010150298', commit_message='End of training', commit_description='', oid='2da3effe0f81d516c3ea7320f4189d4010150298', pr_url=None, pr_revision=None, pr_num=None)

你现在可以与你的所有朋友、家人、最喜欢的宠物共享这个模型:他们都可以使用标识符`"your-username/the-name-you-picked"`来加载它，例如:

```python
从转换器导入AutoModelForSeq2SeqLM

model = automodelforseq2seqlm .from_pre - trained("sgugger/my-awesome-model")
```

## Inference

很好，现在你已经对模型进行了微调，你可以使用它进行推理!

想出一些你想翻译成另一种语言的文本。对于T5，您需要根据您正在处理的任务为输入添加前缀。为了从英语翻译到中文，你应该像下面这样在输入前加上前缀:

In [119]:
text = ["hello.","translate English to Chinese: Legumes share resources with nitrogen-fixing bacteria."]

尝试用于推理的微调模型的最简单方法是在[pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline)中使用它。用你的模型实例化一个用于翻译的`pipeline`，并将你的文本传递给它:

In [120]:
from transformers import pipeline

checkpoint = "weege007/opus-mt-en-zh-finetuned-en-to-zh"

#translator = pipeline("translation_en_to_zh", model=checkpoint, max_length=1024)
translator = pipeline("translation", model=checkpoint, max_length=1024)
translator(text)

[{'translation_text': '哈罗。'}, {'translation_text': '英文译为中文:Legames与固氮细菌共享资源。'}]

如果你愿意，你也可以手动复制`pipeline`的结果:

将文本分词并将`input_ids`作为PyTorch张量返回:

In [122]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# 注意如果是多个文本，长度不一样，需要padding, truncation
inputs = tokenizer(text,  return_tensors="pt", padding=True, truncation=True).input_ids
inputs

tensor([[23675,     6,     0, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000],
        [15955,  3287,     9,  6019,    37, 34696, 47055,    24,  2064,   247,
            29, 53004,    16,   594, 10110,   129, 54391,     6,     0]])

使用[generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate)方法创建翻译。有关控制生成的不同文本生成策略和参数的更多详细信息，请查看[text generation](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/text_generation) API。

In [124]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
outputs = model.generate(inputs, max_new_tokens=1024, do_sample=True, top_k=30, top_p=0.95)
outputs

tensor([[65000,  5359,    10,  5359,    10,     0, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000],
        [65000,     8,  6963, 17717,    76, 21121,    37,   233, 25259,  5955,
            24,    67, 18374, 32989, 25381,  9565,   744,    10,     0]])

将生成的token id解码为文本:

In [126]:
tokenizer.decode(outputs[0], skip_special_tokens=True)
tokenizer.decode(outputs[1], skip_special_tokens=True)

'英文译为中文:Legames与固氮细菌共享资源。'

In [127]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['你好。 你好。', '英文译为中文:Legames与固氮细菌共享资源。']